# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import NOC_tools, NOC_shoreline, NOC_download, NOC_preprocess,\
    SDS_tools, SDS_shoreline, SDS_download, SDS_preprocess

settings ={'output_epsg': 32737,
           'reference_shoreline': True,
           'max_dist_ref': 100,        # max distance (in meters) allowed from the reference shoreline
           'min_length_sl': 500,       # min length (in meters) for shorelines
          }

In [2]:
# directory where the data will be accessed and stored
dataPartition = 'c:\\data\\coastsat'
country = 'kenya'
base_filepath = os.path.join(dataPartition, country)  

# filepaths etc
filepath_sites = os.path.join(base_filepath, 'sites')
sites = os.listdir(filepath_sites)

sat_name = 'S1'

dates = ['2020-01-01', '2020-06-30']

for site in sites:

    kml_filepath = os.path.join(filepath_sites, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)
    
    sitename = site[:site.find('.')]

    inputs = {'polygon': roi_polygon, 'dates': dates, 'sat_list': [sat_name],
              'sitename': sitename, 'filepath': base_filepath, 'pixel_size': 10,
              'polarisation': 'VH', 'sigma': 5}
    
    settings['inputs'] = inputs
    
    print(f'Processing: {sitename}')

Processing: Watamu


In [3]:
    #metadata = NOC_download.retrieve_median_sar(inputs)
    ## load saved metadata
    metadata = NOC_download.get_metadata(inputs)
    
    print('finished ...')

finished ...


In [4]:
    settings['reference_shoreline'] = NOC_preprocess.get_reference_shoreline(inputs)

In [ ]:
    %matplotlib qt
    
    ##Batch shoreline detection
    output = NOC_shoreline.extract_shorelines_sar(metadata, settings)
    
    print('finished ...')

Mapping shorelines:
S1:   100%



In [ ]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

In [ ]:
gdf = SDS_tools.output_to_gdf(output, 'lines')

satname = output['satname'][0]
file_string = f'{sitename}_shorelines_{inputs["polarisation"]}.geojson'

if  ~gdf.empty:
    gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])} # set layer projection
    # save GEOJSON layer to file
    gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'],
                             file_string),
                             driver='GeoJSON', encoding='utf-8')
    print(f'written: {file_string}')
else:
    print('no shorelines to be seen ...')